In [1]:
import phenix_mgt as phnx
import pandas as pd

In [2]:
# Do this only once to save SQL table into CSV files
#phnx.extract_Phenix_SQLtable2CSV(path='./Phenix_DB/')

## Load raw Phenix data

In [3]:
# List of Phenix tables
lst_table = ['CategorieProduits',
            'CommandeProduits',
            'Commandes',
            'Comptes',
            'OffreProduits',
            'Offres',
            'Produits']

In [4]:
df_catP = phnx.get_CategorieProduits('./Phenix_DB/' + lst_table[0] + '.csv')
df_CP   = phnx.get_CommandeProduits ('./Phenix_DB/' + lst_table[1] + '.csv')
df_CO   = phnx.get_Commandes        ('./Phenix_DB/' + lst_table[2] + '.csv')
df_EC   = phnx.get_Comptes          ('./Phenix_DB/' + lst_table[3] + '.csv', prefix='EC_')
df_RC   = phnx.get_Comptes          ('./Phenix_DB/' + lst_table[3] + '.csv', prefix='RC_')
df_OP   = phnx.get_OffreProduits    ('./Phenix_DB/' + lst_table[4] + '.csv')
df_O    = phnx.get_Offres           ('./Phenix_DB/' + lst_table[5] + '.csv')
df_P    = phnx.get_Produits         ('./Phenix_DB/' + lst_table[6] + '.csv')

In [5]:
print('--DATAFRAME SIZES--')
print('Commande : %d' % len(df_CO))
print('Com prod : %d' % len(df_CP))
print('Cat prod : %d' % len(df_catP))
print('Offres   : %d' % len(df_O))
print('Off prod : %d' % len(df_OP))
print('Produits : %d' % len(df_P))
print('E_Compte : %d' % len(df_EC))
print('R_Compte : %d' % len(df_RC))

--DATAFRAME SIZES--
Commande : 64945
Com prod : 3676721
Cat prod : 20
Offres   : 65522
Off prod : 3651709
Produits : 946612
E_Compte : 2519
R_Compte : 2519


## Prepare the tables

In [ ]:
## Cleaning Comptes table
df_matrix = phnx.geotag_Compte(df_EC,prefix='EC_',IGN_API='')
df_EC = phnx.replace_geotag(df_EC,df_matrix,prefix='EC_', threshold=1)
df_RC = phnx.replace_geotag(df_RC,df_matrix,prefix='RC_', threshold=1)

## Join the tables

In [50]:
dict_merge = [{'df2':df_CO, 'on_col': ['Id','CO_Id'],                   'col_name':'CO_Id', 'how':'right', 'del_col':False},
              {'df2':df_RC, 'on_col': ['Recepteur_Id','RC_Id'],         'col_name':'temp_Id', 'how':'left', 'del_col':True},
              {'df2':df_O,  'on_col': ['Offre_Id','O_Id'],              'col_name':'temp_Id', 'how':'left', 'del_col':True},
              {'df2':df_EC, 'on_col': ['Compte_Id','EC_Id'],             'col_name':'temp_Id', 'how':'left', 'del_col':True},]

df_Phenix = phnx.merge_loop_tables(dict_merge)

----> STEP ---->
['CO_Id', 'CO_DateCommande', 'CO_EstReceptionne', 'CO_Statut', 'Offre_Id', 'Recepteur_Id']
64945
----> STEP ---->
['CO_Id', 'CO_DateCommande', 'CO_EstReceptionne', 'CO_Statut', 'Offre_Id', 'RC_Adresse_AdresseLigne1', 'RC_Adresse_AdresseLigne2', 'RC_Adresse_AdresseLigne3', 'RC_Adresse_CodePostal', 'RC_Adresse_Ville', 'RC_Latitude', 'RC_Longitude', 'RC_Nom', 'RC_TypeCompte']
64945
----> STEP ---->
['CO_Id', 'CO_DateCommande', 'CO_EstReceptionne', 'CO_Statut', 'RC_Adresse_AdresseLigne1', 'RC_Adresse_AdresseLigne2', 'RC_Adresse_AdresseLigne3', 'RC_Adresse_CodePostal', 'RC_Adresse_Ville', 'RC_Latitude', 'RC_Longitude', 'RC_Nom', 'RC_TypeCompte', 'Compte_Id']
64945
----> STEP ---->
['CO_Id', 'CO_DateCommande', 'CO_EstReceptionne', 'CO_Statut', 'RC_Adresse_AdresseLigne1', 'RC_Adresse_AdresseLigne2', 'RC_Adresse_AdresseLigne3', 'RC_Adresse_CodePostal', 'RC_Adresse_Ville', 'RC_Latitude', 'RC_Longitude', 'RC_Nom', 'RC_TypeCompte', 'EC_Adresse_AdresseLigne1', 'EC_Adresse_AdresseL

In [58]:
# Liste des Commandes ayant des comptes avec le mauvais type
# TypeCompte 1: Emetteur
# TypeCompte 2: Recepteur
df_Phenix[(df_Phenix.EC_TypeCompte == 2) | (df_Phenix.RC_TypeCompte == 1)]

,CO_Id,CO_DateCommande,CO_EstReceptionne,CO_Statut,RC_Adresse_AdresseLigne1,RC_Adresse_AdresseLigne2,RC_Adresse_AdresseLigne3,RC_Adresse_CodePostal,RC_Adresse_Ville,RC_Latitude,...,RC_TypeCompte,EC_Adresse_AdresseLigne1,EC_Adresse_AdresseLigne2,EC_Adresse_AdresseLigne3,EC_Adresse_CodePostal,EC_Adresse_Ville,EC_Latitude,EC_Longitude,EC_Nom,EC_TypeCompte
10,15,2015-09-04 12:34:33.777,False,1,NaN,NaN,7 place du fer à cheval,31000,TOULOUSE,43.606754,...,2,Appartement 400,NaN,"6, 7 place du fer à cheval",31300,Toulouse,43.616284,1.412382,SO PHENIX,2
113,298,2015-11-20 18:54:42.923,False,1,NaN,NaN,261 avenue du 8 mai 1945,93330,Neuilly-sur-Marne,48.865682,...,2,NaN,NaN,261 avenue du 8 mai 1945,93330,Neuilly-sur-Marne,48.865682,2.534231,CAJN (Collectif Adultes et Jeunes Nocéens),2
114,299,2015-11-20 18:55:23.763,False,1,NaN,NaN,84 Rue du Mont Blanc,77176,Savigny-le-Temple,48.604535,...,2,NaN,NaN,261 avenue du 8 mai 1945,93330,Neuilly-sur-Marne,48.865682,2.534231,CAJN (Collectif Adultes et Jeunes Nocéens),2
115,300,2015-11-20 18:56:37.147,False,1,NaN,NaN,7 rue Jean Baptite Lebas,62710,Courrières,50.451929,...,2,NaN,NaN,261 avenue du 8 mai 1945,93330,Neuilly-sur-Marne,48.865682,2.534231,CAJN (Collectif Adultes et Jeunes Nocéens),2
116,301,2015-11-20 18:57:35.857,False,1,NaN,NaN,32 Allée Pierre de Montereau,93150,Le Blanc Mesnil,48.948205,...,2,NaN,NaN,261 avenue du 8 mai 1945,93330,Neuilly-sur-Marne,48.865682,2.534231,CAJN (Collectif Adultes et Jeunes Nocéens),2
117,302,2015-11-20 18:58:10.713,False,1,NaN,NaN,19 rue Petit de Beauverger,77170,Brie-Comte-Robert,48.694178,...,2,NaN,NaN,261 avenue du 8 mai 1945,93330,Neuilly-sur-Marne,48.865682,2.534231,CAJN (Collectif Adultes et Jeunes Nocéens),2
155,365,2015-12-16 10:45:28.770,True,4,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872942,...,2,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872782,2.371196,PHENIX Association,2
168,384,2016-01-06 16:02:31.060,False,1,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872782,...,2,NaN,NaN,15 rue Martel,75010,Paris,48.874269,2.352848,Association Démo,2
252,563,2016-02-26 11:48:34.723,False,1,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872942,...,2,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872782,2.371196,PHENIX Association,2
426,930,2016-05-10 10:46:11.933,False,1,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872942,...,2,NaN,NaN,208 rue Saint-Maur,75010,PARIS,48.872782,2.371196,PHENIX Association,2
